In [1]:
"""Multilayer Perceptron for drug response problem"""

from __future__ import division, print_function

import argparse
import csv
import logging
import sys

import numpy as np
import pandas as pd

from keras import backend as K
from keras import metrics
from sklearn.metrics import r2_score, mean_squared_error
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Dense, Dropout, LocallyConnected1D, Conv1D, MaxPooling1D, Flatten, Conv2D, LocallyConnected2D
from keras.callbacks import Callback, ModelCheckpoint, ProgbarLogger

# For non-interactive plotting
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt


#import p1b3 as benchmark
import p1b3_all_drugs_new_data as benchmark
import candle

sys.argv = [''] # for Jupyter nbs

#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


Importing candle utils for keras


In [2]:
def rmse(y_true, y_pred):
    return K.sqrt(mean_squared_error(y_true, y_pred))

def initialize_parameters(default_model = 'p1b3_default_model.txt'):

    # Build benchmark object
    p1b3Bmk = benchmark.BenchmarkP1B3(benchmark.file_path, default_model, 'keras',
    prog='p1b3_baseline', desc='Multi-task (DNN) for data extraction from clinical reports - Pilot 3 Benchmark 1')
    
    # Initialize parameters
    gParameters = candle.finalize_parameters(p1b3Bmk)
    #benchmark.logger.info(a'Params: {}'.format(gParameters))

    return gParameters

def str2lst(string_val):
    result = [int(x) for x in string_val.split(' ')]
    return result


def evaluate_keras_metric(y_true, y_pred, metric):
    objective_function = metrics.get(metric)
    objective = objective_function(y_true, y_pred)
    return K.eval(objective)


def evaluate_model(model, generator, steps, metric, category_cutoffs=[0.]):
    y_true, y_pred = None, None
    count = 0
    while count < steps:
        x_batch, y_batch = next(generator)
        y_batch_pred = model.predict_on_batch(x_batch)
        y_batch_pred = y_batch_pred.ravel()
        y_true = np.concatenate((y_true, y_batch)) if y_true is not None else y_batch
        y_pred = np.concatenate((y_pred, y_batch_pred)) if y_pred is not None else y_batch_pred
        count += 1

    loss = evaluate_keras_metric(y_true.astype(np.float32), y_pred.astype(np.float32), metric)

    y_true_class = np.digitize(y_true, category_cutoffs)
    y_pred_class = np.digitize(y_pred, category_cutoffs)

    # theano does not like integer input
    acc = evaluate_keras_metric(y_true_class.astype(np.float32), y_pred_class.astype(np.float32), 'binary_accuracy')  # works for multiclass labels as well

    return loss, acc, y_true, y_pred, y_true_class, y_pred_class


def plot_error(y_true, y_pred, batch, file_ext, file_pre='output_dir', subsample=1000):
    if batch % 10:
        return

    total = len(y_true)
    if subsample and subsample < total:
        usecols = np.random.choice(total, size=subsample, replace=False)
        y_true = y_true[usecols]
        y_pred = y_pred[usecols]

    y_true = y_true * 100
    y_pred = y_pred * 100
    diffs = y_pred - y_true

    bins = np.linspace(-200, 200, 100)
    if batch == 0:
        y_shuf = np.random.permutation(y_true)
        plt.hist(y_shuf - y_true, bins, alpha=0.5, label='Random')

    #plt.hist(diffs, bins, alpha=0.35-batch/100., label='Epoch {}'.format(batch+1))
    plt.hist(diffs, bins, alpha=0.3, label='Epoch {}'.format(batch+1))
    plt.title("Histogram of errors in percentage growth")
    plt.legend(loc='upper right')
    plt.savefig(file_pre+'.histogram'+file_ext+'.b'+str(batch)+'.png')
    plt.close()

    # Plot measured vs. predicted values
    fig, ax = plt.subplots()
    plt.grid('on')
    ax.scatter(y_true, y_pred, color='red', s=10)
    ax.plot([y_true.min(), y_true.max()],
            [y_true.min(), y_true.max()], 'k--', lw=4)
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    plt.savefig(file_pre+'.diff'+file_ext+'.b'+str(batch)+'.png')
    plt.close()


class MyLossHistory(Callback):
    def __init__(self, progbar, val_gen, test_gen, val_steps, test_steps, metric, category_cutoffs=[0.], ext='', pre='save'):
        super(MyLossHistory, self).__init__()
        self.progbar = progbar
        self.val_gen = val_gen
        self.test_gen = test_gen
        self.val_steps = val_steps
        self.test_steps = test_steps
        self.metric = metric
        self.category_cutoffs = category_cutoffs
        self.pre = pre
        self.ext = ext

    def on_train_begin(self, logs={}):
        self.best_val_loss = np.Inf
        self.best_val_acc = -np.Inf

    def on_epoch_end(self, batch, logs={}):
        val_loss, val_acc, y_true, y_pred, y_true_class, y_pred_class = evaluate_model(self.model, self.val_gen, self.val_steps, self.metric, self.category_cutoffs)
        test_loss, test_acc, _, _, _, _ = evaluate_model(self.model, self.test_gen, self.test_steps, self.metric, self.category_cutoffs)
        self.progbar.append_extra_log_values([('val_acc', val_acc), ('test_loss', test_loss), ('test_acc', test_acc)])
        if float(logs.get('val_loss', 0)) < self.best_val_loss:
            plot_error(y_true, y_pred, batch, self.ext, self.pre)
        self.best_val_loss = min(float(logs.get('val_loss', 0)), self.best_val_loss)
        self.best_val_acc = max(float(logs.get('val_acc', 0)), self.best_val_acc)


class MyProgbarLogger(ProgbarLogger):
    def __init__(self, samples):
        super(MyProgbarLogger, self).__init__(count_mode='samples')
        self.samples = samples

    def on_train_begin(self, logs=None):
        super(MyProgbarLogger, self).on_train_begin(logs)
        self.verbose = 1
        self.extra_log_values = []
        self.params['samples'] = self.samples

    def on_batch_begin(self, batch, logs=None):
        if self.seen < self.target:
            self.log_values = []
            self.extra_log_values = []

    def append_extra_log_values(self, tuples):
        for k, v in tuples:
            self.extra_log_values.append((k, v))

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        epoch_log = 'Epoch {}/{}'.format(epoch + 1, self.epochs)
        for k in self.params['metrics']:
            if k in logs:
                self.log_values.append((k, logs[k]))
                epoch_log += ' - {}: {:.4f}'.format(k, logs[k])
        for k, v in self.extra_log_values:
            self.log_values.append((k, v))
            epoch_log += ' - {}: {:.4f}'.format(k, float(v))
        if self.verbose:
            self.progbar.update(self.seen, self.log_values)
        benchmark.logger.debug(epoch_log)

def add_conv_layer(model, layer_params, input_dim=None, locally_connected=False):
    if len(layer_params) == 3: # 1D convolution
        filters = layer_params[0]
        filter_len = layer_params[1]
        stride = layer_params[2]
        if locally_connected:
            if input_dim:
                model.add(LocallyConnected1D(filters, filter_len, strides=stride, input_shape=(input_dim, 1)))
            else:
                model.add(LocallyConnected1D(filters, filter_len, strides=stride))
        else:
            if input_dim:
                model.add(Conv1D(filters, filter_len, strides=stride, input_shape=(input_dim, 1)))
            else:
                model.add(Conv1D(filters, filter_len, strides=stride))
    elif len(layer_params) == 5: # 2D convolution
        filters = layer_params[0]
        filter_len = (layer_params[1], layer_params[2])
        stride = (layer_params[3], layer_params[4])
        if locally_connected:
            if input_dim:
                model.add(LocallyConnected2D(filters, filter_len, strides=stride, input_shape=(input_dim, 1)))
            else:
                model.add(LocallyConnected2D(filters, filter_len, strides=stride))
        else:
            if input_dim:
                model.add(Conv2D(filters, filter_len, strides=stride, input_shape=(input_dim, 1)))
            else:
                model.add(Conv2D(filters, filter_len, strides=stride))
    return model


In [3]:
gParameters = initialize_parameters()
gParameters['epochs'] = 5
gParameters['train_steps'] = 100
gParameters['val_steps'] = 10
gParameters['test_steps'] = 10
benchmark.check_params(gParameters)

Params:
{'activation': 'relu',
 'batch_normalization': False,
 'batch_size': 100,
 'category_cutoffs': [0.0],
 'cell_features': ['expression'],
 'cell_noise_sigma': 0.0,
 'data_type': <class 'numpy.float32'>,
 'dense': [1000, 500, 100, 50],
 'dropout': 0.1,
 'drug_features': ['descriptors'],
 'epochs': 50,
 'experiment_id': 'EXP000',
 'feature_subsample': 0,
 'initialization': 'normal',
 'learning_rate': 0.001,
 'logfile': None,
 'loss': 'mse',
 'max_logconc': -4.0,
 'min_logconc': -5.0,
 'optimizer': 'sgd',
 'output_dir': '/lustre/schandra_crpl/users/2216/NCI-DOE-Collab-Pilot1-Single-Drug-Response-Predictor/Pilot1/P1B3/save/EXP000/RUN000',
 'profiling': False,
 'rng_seed': 2017,
 'run_id': 'RUN000',
 'scaling': 'std',
 'scramble': False,
 'shuffle': False,
 'subsample': 'naive_balancing',
 'test_cell_split': 0.15,
 'timeout': -1,
 'train_bool': True,
 'val_split': 0.1,
 'verbose': None,
 'workers': 1}


In [4]:
"""
Runs the model using the specified set of parameters

Args:
   gParameters: a python dictionary containing the parameters (e.g. epoch)
   to run the model with.
"""
#
if 'dense' in gParameters:
    dval = gParameters['dense']
    if type(dval) != list:
        res = list(dval)
    #try:
        #is_str = isinstance(dval, basestring)
    #except NameError:
        #is_str = isinstance(dval, str)
    #if is_str:
        #res = str2lst(dval)
        gParameters['dense'] = res
    print(gParameters['dense'])

if 'conv' in gParameters:
    flat = gParameters['conv']
    gParameters['conv'] = [flat[i:i+3] for i in range(0, len(flat), 3)]
    #conv_list = p1_common.parse_conv_list(gParameters['conv'])
    #cval = gParameters['conv']
    #try:
    #    is_str = isinstance(cval, basestring)
    #except NameError:
    #    is_str = isinstance(cval, str)
    #if is_str:
    #    res = str2lst(cval)
    #    gParameters['conv'] = res
    print('Conv input', gParameters['conv'])
# print('Params:', gParameters)
# Construct extension to save model
ext = benchmark.extension_from_parameters(gParameters, '.keras')
logfile = gParameters['logfile'] if gParameters['logfile'] else gParameters['output_dir']+ext+'.log'

fh = logging.FileHandler(logfile)
fh.setFormatter(logging.Formatter("[%(asctime)s %(process)d] %(message)s", datefmt="%Y-%m-%d %H:%M:%S"))
fh.setLevel(logging.DEBUG)

sh = logging.StreamHandler()
sh.setFormatter(logging.Formatter(''))
sh.setLevel(logging.DEBUG if gParameters['verbose'] else logging.INFO)

benchmark.logger.setLevel(logging.DEBUG)
benchmark.logger.addHandler(fh)
benchmark.logger.addHandler(sh)
benchmark.logger.info('Params: {}'.format(gParameters))

# Get default parameters for initialization and optimizer functions
kerasDefaults = candle.keras_default_config()
seed = gParameters['rng_seed']

# Build dataset loader object
loader = benchmark.DataLoader(seed=seed, dtype=gParameters['data_type'],
                         val_split=gParameters['val_split'],
                         test_cell_split=gParameters['test_cell_split'],
                         cell_features=gParameters['cell_features'],
                         drug_features=gParameters['drug_features'],
                         feature_subsample=gParameters['feature_subsample'],
                         scaling=gParameters['scaling'],
                         scramble=gParameters['scramble'],
                         min_logconc=gParameters['min_logconc'],
                         max_logconc=gParameters['max_logconc'],
                         subsample=gParameters['subsample'],
                         category_cutoffs=gParameters['category_cutoffs'])

# Initialize weights and learning rule
initializer_weights = candle.build_initializer(gParameters['initialization'], kerasDefaults, seed)
initializer_bias = candle.build_initializer('constant', kerasDefaults, 0.)

gParameters['learning_rate'] = 0.01
gParameters['activation'] = 'tanh'
activation = gParameters['activation']



# Define model architecture
gen_shape = None
out_dim = 1

model = Sequential()
if 'dense' in gParameters: # Build dense layers
    for layer in gParameters['dense']:
        if layer:
            model.add(Dense(layer, input_dim=loader.input_dim,
                        kernel_initializer=initializer_weights,
                        bias_initializer=initializer_bias))
            if gParameters['batch_normalization']:
                model.add(BatchNormalization())
            model.add(Activation(gParameters['activation']))
            if gParameters['dropout']:
                model.add(Dropout(gParameters['dropout']))
else: # Build convolutional layers
    gen_shape = 'add_1d'
    layer_list = list(range(0, len(gParameters['conv'])))
    lc_flag=False
    if 'locally_connected' in gParameters:
        lc_flag = True

    for l, i in enumerate(layer_list):
        if i == 0:
            add_conv_layer(model, gParameters['conv'][i], input_dim=loader.input_dim,locally_connected=lc_flag)
        else:
            add_conv_layer(model, gParameters['conv'][i],locally_connected=lc_flag)
        if gParameters['batch_normalization']:
                model.add(BatchNormalization())
        model.add(Activation(gParameters['activation']))
        if gParameters['pool']:
            model.add(MaxPooling1D(pool_size=gParameters['pool']))
    model.add(Flatten())

model.add(Dense(out_dim))

# Define optimizer
optimizer = candle.build_optimizer(gParameters['optimizer'],
                                            gParameters['learning_rate'],
                                            kerasDefaults)

#gParameters['loss'] = rmse
# Compile and display model
model.compile(loss=gParameters['loss'], optimizer=optimizer)
model.summary()
benchmark.logger.debug('Model: {}'.format(model.to_json()))

train_gen = benchmark.DataGenerator(loader, batch_size=gParameters['batch_size'], shape=gen_shape, name='train_gen', cell_noise_sigma=gParameters['cell_noise_sigma']).flow()
val_gen = benchmark.DataGenerator(loader, partition='val', batch_size=gParameters['batch_size'], shape=gen_shape, name='val_gen').flow()
val_gen2 = benchmark.DataGenerator(loader, partition='val', batch_size=gParameters['batch_size'], shape=gen_shape, name='val_gen2').flow()
test_gen = benchmark.DataGenerator(loader, partition='test', batch_size=gParameters['batch_size'], shape=gen_shape, name='test_gen').flow()

train_steps = int(loader.n_train/gParameters['batch_size'])
val_steps = int(loader.n_val/gParameters['batch_size'])
test_steps = int(loader.n_test/gParameters['batch_size'])

if 'train_steps' in gParameters:
    train_steps = gParameters['train_steps']
if 'val_steps' in gParameters:
    val_steps = gParameters['val_steps']
if 'test_steps' in gParameters:
    test_steps = gParameters['test_steps']

checkpointer = ModelCheckpoint(filepath=gParameters['output_dir']+'.model'+ext+'.h5', save_best_only=True)
progbar = MyProgbarLogger(train_steps * gParameters['batch_size'])
loss_history = MyLossHistory(progbar=progbar, val_gen=val_gen2, test_gen=test_gen,
                        val_steps=val_steps, test_steps=test_steps,
                        metric=gParameters['loss'], category_cutoffs=gParameters['category_cutoffs'],
                        ext=ext, pre=gParameters['output_dir'])



Params: {'dense': [1000, 500, 100, 50], 'batch_size': 100, 'epochs': 5, 'activation': 'relu', 'loss': 'mse', 'optimizer': 'sgd', 'learning_rate': 0.001, 'scaling': 'std', 'dropout': 0.1, 'feature_subsample': 0, 'val_split': 0.1, 'rng_seed': 2017, 'initialization': 'normal', 'min_logconc': -5.0, 'max_logconc': -4.0, 'category_cutoffs': [0.0], 'test_cell_split': 0.15, 'cell_features': ['expression'], 'drug_features': ['descriptors'], 'subsample': 'naive_balancing', 'batch_normalization': False, 'cell_noise_sigma': 0.0, 'output_dir': '/lustre/schandra_crpl/users/2216/NCI-DOE-Collab-Pilot1-Single-Drug-Response-Predictor/Pilot1/P1B3/save/EXP000/RUN000', 'verbose': None, 'logfile': None, 'train_bool': True, 'experiment_id': 'EXP000', 'run_id': 'RUN000', 'shuffle': False, 'profiling': False, 'scramble': False, 'workers': 1, 'data_type': <class 'numpy.float32'>, 'timeout': -1, 'train_steps': 100, 'val_steps': 10, 'test_steps': 10}


[1000, 500, 100, 50]


/home/2216/.conda/envs/Combo/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2878: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/2216/.local/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/lustre/schandra_crpl/users/2216/NCI-DOE-Collab-Pilot1-Single-Drug-Response-Predictor/Pilot1/P1B3/p1b3_all_drugs_new_data.py:620: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nci60_filter

       CELLNAME     AUC
NSC                    
1          7860  0.9258
1          7860  0.9154
1          7860  0.9162
100044     7860  0.9916
100055     7860  0.7956
NCI exp dtypes:
CELLNAME     object
AARS        float64
ABCB6       float64
ABCC5       float64
ABCF1       float64
             ...   
ZNF395      float64
ZNF451      float64
ZNF586      float64
ZNF589      float64
ZW10        float64
Length: 955, dtype: object


Distribution of dose response:
                AUC
count  1.433472e+06
mean   9.128660e-01
std    1.112186e-01
min    0.000000e+00
25%    8.762000e-01
50%    9.563000e-01
75%    9.889000e-01
max    1.000000e+00
Category cutoffs: [0.0]
Dose response bin counts:
  Class 0:       0 (0.0000) - between +0.00 and +0.00
  Class 1: 1433472 (1.0000) - between +0.00 and +0.01
  Total:   1433472
Rows in train: 1288887, val: 143209, test: 1376
Input features shapes:
  cell_expression: (954,)
  drug_descriptors: (3809,)
Total input dimensions: 4763


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              4764000   
_________________________________________________________________
activation_1 (Activation)    (None, 1000)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 500)               500500    
_________________________________________________________________
activation_2 (Activation)    (None, 500)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 500)               0         
_______________________________________

In [5]:
# Seed random generator for training
np.random.seed(seed)

candleRemoteMonitor = candle.CandleRemoteMonitor(params=gParameters)

history = model.fit_generator(train_gen, train_steps,
                    epochs=gParameters['epochs'],
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    verbose=0,
                    callbacks=[checkpointer, loss_history, progbar, candleRemoteMonitor],
                    )

benchmark.logger.removeHandler(fh)
benchmark.logger.removeHandler(sh)


Epoch 1/5
10000/10000 [==============================] - 82s 8ms/step - loss: 0.0724 - val_loss: 7.7196e-04 - val_acc: 0.8010 - test_loss: 0.0025 - test_acc: 0.9920
Epoch 2/5
10000/10000 [==============================] - 69s 7ms/step - loss: 0.0157 - val_loss: 0.0012 - val_acc: 0.9770 - test_loss: 0.0013 - test_acc: 1.0000
Epoch 3/5
10000/10000 [==============================] - 69s 7ms/step - loss: 0.0102 - val_loss: 3.2772e-04 - val_acc: 0.3720 - test_loss: 2.3871e-04 - test_acc: 0.2140
Epoch 4/5
10000/10000 [==============================] - 69s 7ms/step - loss: 0.0079 - val_loss: 2.2484e-04 - val_acc: 0.1040 - test_loss: 1.9101e-04 - test_acc: 0.2000
Epoch 5/5
10000/10000 [==============================] - 69s 7ms/step - loss: 0.0065 - val_loss: 1.7414e-05 - val_acc: 0.9540 - test_loss: 7.2104e-06 - test_acc: 1.0000


In [6]:
# validation
val_data = pd.read_csv('~/xgboost-single-drug-reponse-prediction/data/val_data_nci60.csv', sep=',')
val_data = val_data.dropna()

val_auc = val_data['AUC'].to_numpy()
val_drugs = val_data['NSC']

val_test = val_data.drop(['SOURCE', 'CELLNAME', 'NSC', 'AUC'], 1)
val_test = val_test.to_numpy()


In [7]:
y_pred = model.predict(val_test)
r_squared = r2_score(val_auc, y_pred)
print("R-squared: ", r_squared)

mse = mean_squared_error(val_auc, y_pred)
rmse = np.sqrt(mse)
print("RMSE: ", rmse)

R-squared:  -29.9289960268995
RMSE:  0.8143571424352765


In [8]:
val_data.head()

,SOURCE,CELLNAME,NSC,AUC,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,...,DLS_01,DLS_02,DLS_03,DLS_04,DLS_05,DLS_06,DLS_07,DLS_cons,LLS_01,LLS_02
769,NCI60,HOP62,141993,0.9161,0.207705,-0.104068,-0.219154,0.089780,-0.133923,0.057540,...,1.0,0.83,1.0,0.6,0.5,0.83,1.0,0.82,0.5,0.88
770,NCI60,MALME3M,141993,0.8999,-0.152366,-0.011850,-0.000318,-0.026186,0.055203,0.253913,...,1.0,0.83,1.0,0.6,0.5,0.83,1.0,0.82,0.5,0.88
771,NCI60,MCF7,141993,0.9327,-0.044345,0.095241,0.017781,0.027932,0.087492,-0.642860,...,1.0,0.83,1.0,0.6,0.5,0.83,1.0,0.82,0.5,0.88
772,NCI60,RPMI8226,141993,0.8790,0.198703,-0.461040,-0.171438,0.085915,0.022913,-0.492307,...,1.0,0.83,1.0,0.6,0.5,0.83,1.0,0.82,0.5,0.88
773,NCI60,SR,141993,0.9267,-0.210877,-0.187361,-0.069424,0.167091,0.313520,-0.394120,...,1.0,0.83,1.0,0.6,0.5,0.83,1.0,0.82,0.5,0.88


In [9]:
nci60_auc_pred = pd.DataFrame(val_data['CELLNAME'])
nci60_auc_pred.insert(1, 'AUC_pred', y_pred)
nci60_auc_pred.insert(1, 'DRUG', val_drugs)
nci60_auc_pred.insert(2, 'AUC_true', val_auc)
nci60_auc_pred

,CELLNAME,DRUG,AUC_true,AUC_pred
769,HOP62,141993,0.9161,0.019326
770,MALME3M,141993,0.8999,0.019326
771,MCF7,141993,0.9327,0.019326
772,RPMI8226,141993,0.8790,0.019326
773,SR,141993,0.9267,0.019326
...,...,...,...,...
2426,SR,57197,0.9983,0.015938
2442,HOP62,633782,0.9139,0.010873
2443,HOP92,633782,0.3732,0.010873
2444,RPMI8226,633782,0.8685,0.010873
